# GSM-VED: Visco-elasto-damage 

In [ ]:
%matplotlib widget
from bmcs_matmod.gsm import GSMMPDP
from bmcs_matmod.gsm.gsm_models import GSMVED1D
import matplotlib.pylab as plt
import sympy as sp
import numpy as np
sp.init_printing()

In [ ]:
gsm = GSMVED1D()
gsm_ved = gsm.F_engine

In [ ]:
(dot_lam_0), _, _ = gsm_ved.H_Lam
gsm_ved.phi_

In [ ]:
gsm_ved.Sig_, gsm_ved.phi_.diff(gsm_ved.Sig.as_explicit()), gsm_ved.Y_, gsm_ved.Phi_

In [ ]:
(gamma_mech, L_, dL_dS_, dL_dS_A_, dR_dA_n1), (eps_n, delta_eps, Eps_n, delta_A, delta_t, Ox, Ix), Sig_n1, f_n1, R_n1, dR_dA_n1_OI = gsm_ved.Sig_f_R_dR_n1


In [ ]:
gamma_mech

In [ ]:
L_

In [ ]:
Sig_n1

In [ ]:
sp.simplify(dL_dS_)

In [ ]:
dL_dS_A_

In [ ]:
R_n1_subs = R_n1 # .replace(sp.sign, lambda x: 1)
R_n1_subs

In [ ]:
delta_A.T

In [ ]:
dR_dA_n1_subs = dR_dA_n1.replace(sp.sign, lambda x: 1)
dR_dA_n1_subs

In [ ]:
mparams = {gsm.E:1, gsm.eta_ve:1e+2, gsm.c:1, gsm.r:1, gsm.S:0.1, gsm.eps_0:1.0}
n_t = 100
n_I = 1
eps_max = 4
t_t = np.linspace(0, 100, n_t)
eps_n_t = np.linspace(0, eps_max, n_t)
Eps_n_Ib = np.zeros((n_I, gsm_ved.n_Eps), dtype=np.float64)
t_t, eps_ta, Eps_t, Sig_t, iter_t, lam_t, (d_t_t, d_eps_ta) = gsm_ved.get_response(eps_n_t, t_t, mparams, 100, )
# Plot viscoplastic strain and damage along time
eps_t = eps_ta[:, 0]
eps_v_t = Eps_t[:, 0, 0]
omega_t = Eps_t[:, 0, 1]

In [ ]:
fig, (ax_sig, ax_omega) = plt.subplots(1, 2, figsize=(12, 4))

# Plot stress-strain and iterations-strain curves
ax_iter = ax_sig.twinx()
ax_eps_v = ax_omega.twinx()

ax_sig.plot(eps_t, Sig_t[:,:,0], color='cadetblue', label=r'$\varepsilon$')
ax_sig.plot(eps_t-eps_v_t, Sig_t[:,:,0], ls='dashed', color='cadetblue', label=r'$\varepsilon-\varepsilon^\mathrm{ve}$')
ax_sig.set_ylabel(r'$\sigma$', color='darkslategray')
ax_sig.set_xlabel(r'$\varepsilon$')
ax_sig.legend()

ax_iter.plot(eps_t, iter_t, ls='dashed', color='gray')
ax_iter.set_ylabel(r'$k$', color='gray')

ax_omega.plot(t_t, omega_t, label='Damage', color='royalblue')
ax_eps_v.plot(t_t, eps_v_t, label='Viscoplastic Strain', color='orange')

ax_omega.set_xlabel(r'$t$')
ax_omega.set_ylabel(r'$\omega$', color='royalblue')
ax_eps_v.set_ylabel(r'$\varepsilon_\mathrm{ve}$', color='orange')

plt.tight_layout()
plt.show()

In [ ]:
%matplotlib widget
mparams = {gsm.E:1, gsm.eta_ve:1e+2, gsm.c:1, gsm.r:1, gsm.S:0.01, gsm.eps_0:0}
t_, eps_dot_, t_1_, t_2_, t3_ = sp.symbols('t, eps_dot t_1 t_2 t_3', real=True, positive=True)
eps_t_ = sp.Piecewise(
    (eps_dot_ * t_, t_ < t_1_),
    (eps_dot_ * t_1_, t_ < t_2_),
    (eps_dot_ * t_1_ - eps_dot_*(t_ - t_2_), True)
)
get_eps_t = sp.lambdify((t_, eps_dot_, t_1_, t_2_, t3_), eps_t_, 'numpy')
final_time = 300
eps_max = 2.5
dot_eps_t = eps_max / final_time * 3
t_t = np.linspace(0, final_time, 1000)
eps_n_t = get_eps_t(t_t, dot_eps_t, 0.25*final_time, 0.75*final_time, final_time)
fig, ax = plt.subplots(1,1)
ax.plot(t_t, eps_n_t)

In [ ]:

t_t, eps_ta, Eps_t, Sig_t, iter_t, lam_t, (d_t_t, d_eps_ta) = gsm_ved.get_response(eps_n_t, t_t, mparams, 100)
# Plot viscoplastic strain and damage along time
eps_t = eps_ta[:, 0]
eps_v_t, omega_t, z_t = Eps_t[:, 0, :].T
sig_v_t, Y_t, Z_t = Sig_t[:, 0, :].T

In [ ]:
fig, (ax_sig, ax_omega) = plt.subplots(1, 2, figsize=(12, 4))

# Plot stress-strain and iterations-strain curves
ax_iter = ax_sig.twinx()
ax_eps = ax_omega.twinx()

ax_sig.plot(eps_t, sig_v_t, color='cadetblue', label=r'$\varepsilon$')
ax_sig.plot(eps_t-eps_v_t, Sig_t[:,:,0], ls='dashed', color='cadetblue', label=r'$\varepsilon-\varepsilon^\mathrm{ve}$')
ax_sig.set_ylabel(r'$\sigma$', color='darkslategray')
ax_sig.set_xlabel(r'$\varepsilon$')
ax_sig.legend()

ax_iter.plot(eps_t, iter_t, ls='dashed', color='gray')
ax_iter.set_ylabel(r'$k$', color='gray')

ax_omega.plot(t_t, omega_t, label='Damage', color='brown')
ax_eps.plot(t_t, eps_t, label=r'$\varepsilon$', color='cadetblue', lw=1)
ax_eps.plot(t_t, eps_v_t, label=r'$\varepsilon_\mathrm{vp}$', color='cadetblue', ls='dashed', lw=1)
#ax_eps.legend()
ax_omega.set_xlabel(r'$t$')
ax_omega.set_ylabel(r'$\omega$', color='brown')
ax_eps.set_ylabel(r'$\varepsilon$', color='cadetblue')

plt.tight_layout()
plt.show()

In [ ]:
mparams_epd = {E:1, eta_ve:1, c:1, r:1, S:500, eps_0:1.0}
margs = [mparams_epd[p] for p in mparams]
eps_n_I = np.array([0, 1.0, 1.0, -1.0, 0, 0], dtype=np.float64)
d_eps_I = np.array([0.1, -0.1, 0.1, -0.1, 2, -2], dtype=np.float64)
n_I = eps_n_I.shape[0]
d_t = 1
Eps_n_Ib = np.zeros((n_I, gsm_ved.n_Eps), dtype=np.float64)

Eps_n1_Ib, Sig_n1_Ib, lam_I, k_I = gsm_ved.get_state_n1(eps_n_I, d_eps_I, d_t, Eps_n_Ib, 3000, *margs )
Eps_n1_Ib, Sig_n1_Ib, lam_I, k_I